# Data processing/feature engineering

In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

In [65]:
#read clean dataset
data = pd.read_csv('data/dataset_clean.csv')

In [66]:
data

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),6,25,1,1,41,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),1,1,7,3,59,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,11,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),1,1,7,2,44,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),1,1,7,1,51,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97104,443847548,100162476,AfricanAmerican,Male,[70-80),1,3,7,3,51,...,No,Down,No,No,No,No,No,Ch,Yes,>30
97105,443847782,74694222,AfricanAmerican,Female,[80-90),1,4,5,5,33,...,No,Steady,No,No,No,No,No,No,Yes,NO
97106,443854148,41088789,Caucasian,Male,[70-80),1,1,7,1,53,...,No,Down,No,No,No,No,No,Ch,Yes,NO
97107,443857166,31693671,Caucasian,Female,[80-90),2,3,7,10,45,...,No,Up,No,No,No,No,No,Ch,Yes,NO


#### Removing drug features

There are multiple drug features in the dataset that were never prescribed to a patient. We will remove all instances of such features if more than 95% of the patients were not presecribed.

In [67]:
#list of all drug features
drugs = np.array(['metformin',
        'repaglinide',
        'nateglinide',
        'chlorpropamide',
        'glimepiride',
        'acetohexamide',
        'glipizide',
        'glyburide',
        'tolbutamide',
        'pioglitazone',
        'rosiglitazone',
        'acarbose',
        'miglitol',
        'troglitazone',
        'tolazamide',
        'examide',
        'citoglipton',
        'insulin',
        'glyburide-metformin',
        'glipizide-metformin',
        'glimepiride-pioglitazone',
        'metformin-rosiglitazone',
        'metformin-pioglitazone'])

In [68]:
#dropping drug columns if more than 95% of the values are No = not prescribed
dropped_drug_cols = []

for d in drugs:
    counts = data[d].value_counts()
    if (counts['No']>0.95*len(data)):
        dropped_drug_cols.append(d)

#storing a list of features that remain
accepted_drug_cols = np.setdiff1d(drugs, dropped_drug_cols).tolist()

data = data.drop(columns = dropped_drug_cols)


#### Encoding the feature: `race`

In [69]:
data['race'].value_counts()

race
Caucasian          74220
AfricanAmerican    18772
Hispanic            2017
Other               1472
Asian                628
Name: count, dtype: int64

In [70]:
#One-hot encoding manually. Not using get_dummies to control the column name

data['caucasian_ind'] = data['race'].isin(['Caucasian']).astype(int)
data['africanamerican_ind'] = data['race'].isin(['AfricanAmerican']).astype(int)
data['hispanic_ind'] = data['race'].isin(['Hispanic']).astype(int)
data['asian_ind'] = data['race'].isin(['Asian']).astype(int)
data['otherrace_ind'] = data['race'].isin(['Other']).astype(int)

dropped_columns = ['race', 'patient_nbr', 'encounter_id']

#### Encoding the feature: `gender`

In [71]:
data['gender'].value_counts()

gender
Female             52336
Male               44772
Unknown/Invalid        1
Name: count, dtype: int64

In [72]:
#dropping entry with Unkown/invalid gender
data = data.drop(data['gender'].loc[data['gender'] == 'Unknown/Invalid'].index)

data['female_ind'] = data['gender'].isin(['Female']).astype(int)
data['male_ind'] = data['gender'].isin(['Male']).astype(int)

dropped_columns.append('gender')

#### Encoding the feature: `age`

In [73]:
data.age.value_counts()

age
[70-80)     24743
[60-70)     21572
[50-60)     16701
[80-90)     16049
[40-50)      9389
[30-40)      3688
[90-100)     2522
[20-30)      1603
[10-20)       682
[0-10)        159
Name: count, dtype: int64

In [74]:
#Using Tam's code here
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
#it's cool to use LabelEncoder here because our 'age' column actually sorted in ascending 'alphabetical' order
#faircare_pd = pd.DataFrame(faircare)
#this is a choice to use LabelEncoder instead of replacing with a dictionary. We'll use a dictionary for replacing other column entries with numbers
data['age'] = label_encoder.fit_transform(data['age']).astype(int)


#### Encoding the diagnoses

Features `diag_1`, `diag_2`, and `diag_3` contain >800 distinct alphanumeric diagnosis codes (ICD-9). We will group them as follows:
- 001–139: infectious and parasitic diseases
- 140–239: neoplasms
- 240–279: endocrine, nutritional and metabolic diseases, and immunity disorders
- 280–289: diseases of the blood and blood-forming organs
- 290–319: mental disorders
- 320–389: diseases of the nervous system and sense organs
- 390–459: diseases of the circulatory system
- 460–519: diseases of the respiratory system
- 520–579: diseases of the digestive system
- 580–629: diseases of the genitourinary system
- 630–679: complications of pregnancy, childbirth, and the puerperium
- 680–709: diseases of the skin and subcutaneous tissue
- 710–739: diseases of the musculoskeletal system and connective tissue
- 740–759: congenital anomalies
- 760–779: certain conditions originating in the perinatal period
- 780–799: symptoms, signs, and ill-defined conditions
- 800–999: injury and poisoning
- E and V codes: external causes of injury and supplemental classification

Note that there are multiple distinct codes for external causes (E000 - E999) as well as for supplementary factors (V01 - V82), however they will be grouped together.

In [75]:
#coding alpha-numeric codes first
data['d_external_index'] = ( (data['diag_1'].str[0] == 'E') |
                             (data['diag_2'].str[0] == 'E') |
                             (data['diag_3'].str[0] == 'E')).astype(int)

data['d_supplementary_index'] = ((data['diag_1'].str[0] == 'V') |
                                 (data['diag_2'].str[0] == 'V') |
                                 (data['diag_3'].str[0] == 'V')).astype(int)

#no diagnosis
#data['d_nodiag_ind'] = ( (data['diag_1'] == 'No') |
#                         (data['diag_2'] == 'No') |
#                         (data['diag_3'] == 'No')).astype(int)

In [76]:
data['diag_1_n'] = data['diag_1'].apply(pd.to_numeric, errors='coerce')
data['diag_2_n'] = data['diag_2'].apply(pd.to_numeric, errors='coerce')
data['diag_3_n'] = data['diag_3'].apply(pd.to_numeric, errors='coerce')

data['diag_1_n'].dropna()
data['diag_2_n'].dropna()
data['diag_3_n'].dropna()

dropped_columns.extend(['diag_1','diag_2','diag_3'])

In [77]:
#infectious and parasitic diseases
data['d_infect_ind'] = ( ((data['diag_1_n'] >= 1) & (data['diag_1_n'] <= 139)) |
                         ((data['diag_2_n'] >= 1) & (data['diag_2_n'] <= 139)) |
                         ((data['diag_3_n'] >= 1) & (data['diag_3_n'] <= 139)) ).astype(int)

#neoplasms
data['d_neoplasm_ind'] = ( ((data['diag_1_n'] >= 140) & (data['diag_1_n'] <= 239)) |
                           ((data['diag_2_n'] >= 140) & (data['diag_2_n'] <= 239)) |
                           ((data['diag_3_n'] >= 140) & (data['diag_3_n'] <= 239)) ).astype(int)

#endocrine, nutritional and metabolic diseases, and immunity disorders
data['d_endocrine_ind'] = ( ((data['diag_1_n'] >= 240) & (data['diag_1_n'] <= 279)) |
                            ((data['diag_2_n'] >= 240) & (data['diag_2_n'] <= 279)) |
                            ((data['diag_3_n'] >= 240) & (data['diag_3_n'] <= 279)) ).astype(int)

#diseases of the blood and blood-forming organs
data['d_blood_ind'] = ( ((data['diag_1_n'] >= 280) & (data['diag_1_n'] <= 289)) |
                        ((data['diag_2_n'] >= 280) & (data['diag_2_n'] <= 289)) |
                        ((data['diag_3_n'] >= 280) & (data['diag_3_n'] <= 289)) ).astype(int)

#mental disorders
data['d_mental_ind'] = ( ((data['diag_1_n'] >= 290) & (data['diag_1_n'] <= 319)) |
                         ((data['diag_2_n'] >= 290) & (data['diag_2_n'] <= 319)) |
                         ((data['diag_3_n'] >= 290) & (data['diag_3_n'] <= 319)) ).astype(int)

#diseases of the nervous system and sense organs
data['d_nervous_ind'] = ( ((data['diag_1_n'] >= 320) & (data['diag_1_n'] <= 389)) |
                          ((data['diag_2_n'] >= 320) & (data['diag_2_n'] <= 389)) |
                          ((data['diag_3_n'] >= 320) & (data['diag_3_n'] <= 389)) ).astype(int)

#diseases of the circulatory system
data['d_circulatory_ind'] = ( ((data['diag_1_n'] >= 390) & (data['diag_1_n'] <= 459)) |
                              ((data['diag_2_n'] >= 390) & (data['diag_2_n'] <= 459)) |
                              ((data['diag_3_n'] >= 390) & (data['diag_3_n'] <= 459)) ).astype(int)

#diseases of the respiratory system
data['d_respiratory_ind'] = ( ((data['diag_1_n'] >= 460) & (data['diag_1_n'] <= 519)) |
                              ((data['diag_2_n'] >= 460) & (data['diag_2_n'] <= 519)) |
                              ((data['diag_3_n'] >= 460) & (data['diag_3_n'] <= 519)) ).astype(int)

#diseases of the digestive system
data['d_digestive_ind'] = ( ((data['diag_1_n'] >= 520) & (data['diag_1_n'] <= 579)) |
                            ((data['diag_2_n'] >= 520) & (data['diag_2_n'] <= 579)) |
                            ((data['diag_3_n'] >= 520) & (data['diag_3_n'] <= 579)) ).astype(int)

#diseases of the genitourinary system
data['d_genitourinary_ind'] = ( ((data['diag_1_n'] >= 580) & (data['diag_1_n'] <= 629)) |
                                ((data['diag_2_n'] >= 580) & (data['diag_2_n'] <= 629)) |
                                ((data['diag_3_n'] >= 580) & (data['diag_3_n'] <= 629)) ).astype(int)

#complications of pregnancy, childbirth, and the puerperium
data['d_pegnancy_ind'] = ( ((data['diag_1_n'] >= 630) & (data['diag_1_n'] <= 679)) |
                           ((data['diag_2_n'] >= 630) & (data['diag_2_n'] <= 679)) |
                           ((data['diag_3_n'] >= 630) & (data['diag_3_n'] <= 679)) ).astype(int)

#diseases of the skin and subcutaneous tissue
data['d_skin_ind'] = ( ((data['diag_1_n'] >= 680) & (data['diag_1_n'] <= 709)) |
                       ((data['diag_2_n'] >= 680) & (data['diag_2_n'] <= 709)) |
                       ((data['diag_3_n'] >= 680) & (data['diag_3_n'] <= 709)) ).astype(int)

#diseases of the musculoskeletal system and connective tissue
data['d_musculo_ind'] = ( ((data['diag_1_n'] >= 710) & (data['diag_1_n'] <= 739)) |
                          ((data['diag_2_n'] >= 710) & (data['diag_2_n'] <= 739)) |
                          ((data['diag_3_n'] >= 710) & (data['diag_3_n'] <= 739)) ).astype(int)

#congenital anomalies
data['d_congenital_ind'] = ( ((data['diag_1_n'] >= 740) & (data['diag_1_n'] <= 759)) |
                             ((data['diag_2_n'] >= 740) & (data['diag_2_n'] <= 759)) |
                             ((data['diag_3_n'] >= 740) & (data['diag_3_n'] <= 759)) ).astype(int)

#certain conditions originating in the perinatal period
data['d_perinatal_ind'] = ( ((data['diag_1_n'] >= 760) & (data['diag_1_n'] <= 779)) |
                            ((data['diag_2_n'] >= 760) & (data['diag_2_n'] <= 779)) |
                            ((data['diag_3_n'] >= 760) & (data['diag_3_n'] <= 779)) ).astype(int)

#symptoms, signs, and ill-defined conditions
data['d_symptoms_ind'] = ( ((data['diag_1_n'] >= 780) & (data['diag_1_n'] <= 799)) |
                           ((data['diag_2_n'] >= 780) & (data['diag_2_n'] <= 799)) |
                           ((data['diag_3_n'] >= 780) & (data['diag_3_n'] <= 799)) ).astype(int)

#injury and poisoning
data['d_injury_ind'] = ( ((data['diag_1_n'] >= 800) & (data['diag_1_n'] <= 999)) |
                         ((data['diag_2_n'] >= 800) & (data['diag_2_n'] <= 999)) |
                         ((data['diag_3_n'] >= 800) & (data['diag_3_n'] <= 999)) ).astype(int)


dropped_columns.extend(['diag_1_n','diag_2_n','diag_3_n'])

#### Encoding test results

In [78]:
data['A1Cresult'].value_counts()

A1Cresult
No      80694
>8       7884
Norm     4837
>7       3693
Name: count, dtype: int64

In [79]:
data['max_glu_serum'].value_counts()

max_glu_serum
No      92010
Norm     2520
>200     1400
>300     1178
Name: count, dtype: int64

In [80]:
A1c_map = {">8" : 3,
           ">7" : 2,
           "Norm" : 1,
           "No" : 0}

max_glu_serum_map = {">300" : 3,
                     ">200" : 2,
                     "Norm" : 1,
                     "No" : 0}

In [81]:
data['A1Cresult'] = data['A1Cresult'].map(A1c_map)
data['max_glu_serum'] = data['max_glu_serum'].map(max_glu_serum_map)

#### Encoding drug features

In [82]:
accepted_drug_cols

['glimepiride',
 'glipizide',
 'glyburide',
 'insulin',
 'metformin',
 'pioglitazone',
 'rosiglitazone']

_Note_: Making a slight change here. IMO, if a patient was never prescribed a certain drug, we can qualitatively treat that as better than the dosage going down which still implies that a patient is on said drug. Flipping the encoding for No and Down.

In [83]:
drugs_map = {
    'No': 0,
    'Down': 1,
    'Steady': 2,
    'Up': 3
}
for d in accepted_drug_cols:
    data[d] = data[d].map(drugs_map)

#### Encoding diabetes med features

In [84]:
data['change'].value_counts()

change
No    52071
Ch    45037
Name: count, dtype: int64

In [85]:
data['diabetesMed'].value_counts()

diabetesMed
Yes    74873
No     22235
Name: count, dtype: int64

In [86]:
data['dia_med_change_ind'] = ( data['change'] == "Ch" ).astype(int)
data['dia_med_nochange_ind'] = ( data['change'] == "No" ).astype(int)
data['diabetesMed'] = ( data['diabetesMed'] == "Yes" ).astype(int)
dropped_columns.extend(['change'])

In [87]:
data = data.drop(columns=dropped_columns)
data.to_csv("./data/data_processed.csv" , index=False)